In [ ]:
import numpy as np
import trimesh

def project_mesh_to_heightmap(mesh, resolution, x_range, y_range):
    # Create the heightmap (2.5D array) initialized to NaN (no data)
    heightmap = np.full(resolution, np.nan)
    
    # Get mesh vertices and faces
    vertices = mesh.vertices
    faces = mesh.faces
    
    # Define the grid
    x_min, x_max = x_range
    y_min, y_max = y_range
    x_step = (x_max - x_min) / resolution[0]
    y_step = (y_max - y_min) / resolution[1]
    
    # Iterate over each pixel in the heightmap
    for i in range(resolution[0]):
        for j in range(resolution[1]):
            # Calculate the world coordinates of the pixel center
            x_coord = x_min + i * x_step
            y_coord = y_min + j * y_step
            ray_origin = np.array([x_coord, y_coord, np.inf])  # Ray originates above the heightmap
            ray_direction = np.array([0, 0, -1])  # Ray is cast downward (negative Z direction)
            
            # Perform ray-casting to find the intersection with the mesh
            ray = trimesh.ray.Ray(origins=[ray_origin], directions=[ray_direction])
            intersections = mesh.ray.intersects_first(ray_origins=ray.origins, ray_directions=ray.directions)
            
            if intersections[0]:  # If there's an intersection
                intersection_point = intersections[1]  # Get the Z-coordinate of the intersection
                heightmap[i, j] = intersection_point  # Update the heightmap with the height value

    return heightmap

# Load your mesh
mesh = trimesh.load_mesh('path_to_your_mesh.stl')

# Define the resolution and range for the height map
resolution = (500, 500)  # 500x500 grid
x_range = (mesh.bounds[0][0], mesh.bounds[1][0])  # X-range based on mesh bounds
y_range = (mesh.bounds[0][1], mesh.bounds[1][1])  # Y-range based on mesh bounds

# Generate the heightmap
heightmap = project_mesh_to_heightmap(mesh, resolution, x_range, y_range)

# Optionally: Display or save the heightmap
import matplotlib.pyplot as plt
plt.imshow(heightmap, cmap='terrain', origin='lower')
plt.colorbar()
plt.show()